In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

In [116]:
# Read data from csv, excel
data = pd.read_csv(r"C:\Workspace\PredictFootballMatches\PredictSuperLeague\csv\turkish_league.csv")
inferenceInput = pd.read_excel(r"C:\Workspace\PredictFootballMatches\PredictSuperLeague\csv\inferenceInput.xlsx")
print("Read completed...")

Read completed...


In [127]:
# Filter data to create features
turkish_league = data.iloc[:,1:]
turkish_league.Season = turkish_league.Season.replace(2223,23)
turkish_league.Date = turkish_league.Date.astype('datetime64[ns]')
turkish_league.reset_index(drop=True,inplace=True)
turkish_league["Week"] = pd.DatetimeIndex(turkish_league['Date']).isocalendar().reset_index().week.astype(int)

# Remove redundant values
del turkish_league["Div"]
turkish_league.dropna(inplace=True)

turkish_league = turkish_league.drop(columns=["HS","AS","FTR","FTHG","FTAG","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"])
turkish_league.columns =["Season","Date","HomeTeam","AwayTeam","HomeFTGoal","AwayFTGoal","FHResult","HomeBet","AwayBet","DrawBet","Week"]

turkish_league["HomeTotalGoal"] = 0
turkish_league["AwayTotalGoal"] = 0

inferenceInput.Date = inferenceInput.Date.astype('datetime64[ns]')
turkish_league = pd.concat([turkish_league, inferenceInput[turkish_league.columns]],ignore_index=True)

week = []
for season in turkish_league.Season.unique():
    x = 1
    for index in turkish_league[turkish_league.Season == season].Week.index:
        if index == len(turkish_league) - 1:
            break
        if (turkish_league.loc[index]["Week"]) != (turkish_league.loc[index + 1]["Week"]):
            x += 1
        week.append(x)
week.insert(len(turkish_league), x)
turkish_league["Week"] = week

In [133]:
conditions = [(turkish_league.FHResult == "H"),(turkish_league.FHResult == "A"),(turkish_league.FHResult == "D")]
values = [1,2,0]
turkish_league['FHResult'] = np.select(conditions, values, default=turkish_league['FHResult'])

print("Set condition completed...")

# Merge with team values
values = pd.read_excel(r"C:\Workspace\PredictFootballMatches\PredictSuperLeague\csv\kadrodegerleri2023.xlsx")

turkish_league.replace("Buyuksehyr","Basaksehir",inplace=True)
turkish_league.replace("Ad. Demirspor","Adana Demirspor",inplace=True)

mergeHome = pd.merge(turkish_league, values, how="left", left_on=["HomeTeam"], right_on=["Team"])
turkish_league = pd.merge(mergeHome, values, how="left", left_on=["AwayTeam"], right_on=["Team"],suffixes=["HomeTeam","AwayTeam"])

turkish_league = turkish_league.drop(["TeamAwayTeam","TeamHomeTeam","AgeHomeTeam","AgeAwayTeam","marketValueAwayTeam","marketValueHomeTeam"],axis=1).sort_values(by=["Date"])

print("Merge completed...")

Set condition completed...
Merge completed...


In [134]:
# Calculate Parameter
turkish_league.reset_index(inplace=True,drop=True)

homeTeamColumn = "HomeTeam"
awayTeamColumn = "AwayTeam"
matchResultColumn = "FHResult"
HomeGoalsColumn = "HomeFTGoal"
AwayGoalsColumn = "AwayFTGoal"

for ind in turkish_league[turkish_league.Season == 23].index:
    homeScore = 0
    homeScore1 = 0
    awayScore = 0
    awayScore1 = 0
    for ind2 in turkish_league.index[0:ind]:
        if (turkish_league.loc[ind2][homeTeamColumn] == turkish_league.loc[ind][homeTeamColumn]):
            homeScore = homeScore + turkish_league.loc[ind2][HomeGoalsColumn]

        if (turkish_league.loc[ind2][awayTeamColumn] == turkish_league.loc[ind][homeTeamColumn]):
            homeScore1 = homeScore1 + turkish_league.loc[ind2][AwayGoalsColumn]
        
        if (turkish_league.loc[ind2][homeTeamColumn] == turkish_league.loc[ind][awayTeamColumn]):
            awayScore = awayScore + turkish_league.loc[ind2][HomeGoalsColumn]

        if (turkish_league.loc[ind2][awayTeamColumn] == turkish_league.loc[ind][awayTeamColumn]):
            awayScore1 = awayScore1 + turkish_league.loc[ind2][AwayGoalsColumn]

    turkish_league.at[ind,"HomeTotalGoal"] = homeScore + homeScore1
    turkish_league.at[ind,"AwayTotalGoal"] = awayScore + awayScore1

turkish_league.drop(columns=["Date",HomeGoalsColumn,AwayGoalsColumn],inplace=True)

print("Calculate parameter completed...")

Calculate parameter completed...


In [179]:
N = 348
# XGBoost HighLow traning
model = xgb.XGBClassifier(max_depth = 2, n_estimators = 200, learning_rate = 0.01,  n_jobs = -1)
data = pd.get_dummies(turkish_league, columns=['HomeTeam','AwayTeam','Week'] ,dtype=int)
dependantY = "FHResult"
# Prepare Data
X_train = data.loc[:, data.columns != dependantY][:N]
y_train = data[dependantY][:N]

X_test = data.loc[:, data.columns != dependantY][N:]
y_test = data[dependantY][N:]

# XG BOOST regression
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

correct_predictions = 0
for yt, yp in zip(y_test, y_pred):
    if yt == yp:
        correct_predictions += 1

#returns accuracy
correct_predictions / len(y_test)

0.45454545454545453

In [401]:
# XGBoost HighLow traning
model = xgb.XGBClassifier(max_depth = 3, n_estimators = 200, learning_rate = 0.02,  n_jobs = -1, sampling_method='gradient_based')
data = pd.get_dummies(turkish_league, columns=['HomeTeam','AwayTeam','Week'] ,dtype=int)
dependantY = "HighLow"
# Prepare Data
X = data.loc[:, data.columns != dependantY]
y = data[dependantY]

# XG BOOST regression
model.fit(X, y)

# pickle.dump(model, open("AlgorithmHighLow.sav", 'wb'))

# print("Train completed...")

xgb.plot_importance(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)